In [1]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

In [2]:
pip install -q agno

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.0 MB/s eta 0:00:00


In [41]:
import pandas as pd

df = pd.read_csv('kpi.csv', parse_dates=['timestamp'])
df['hour'] = df['timestamp'].dt.hour

# Global storage for DataFrame
_dataframe_store = {'df': None}

In [42]:

from agno.tools import tool

@tool
def get_all_cells_prb_timeseries() -> dict:
    """
    Extracts Downlink PRB utilization values for ALL cells.

    Returns:
        Dictionary containing:
        - all_cells: list of all cell IDs
        - cells_data: dict with each cell's hourly PRB utilization
        - summary_stats: overall statistics
    """
    df = _dataframe_store['df']

    if df is None:
        return {"error": "No DataFrame loaded"}

    all_cells = df['cell_id'].unique().tolist()
    cells_data = {}

    for cell_id in all_cells:
        subset = df[df["cell_id"] == cell_id]

        if not subset.empty:
            prb_by_hour = subset.groupby("hour")["Downlink PRB Utilization"].mean().to_dict()

            cells_data[str(cell_id)] = {
                "prb_by_hour": {int(k): round(float(v), 2) for k, v in prb_by_hour.items()},
                "avg_prb": round(float(subset["Downlink PRB Utilization"].mean()), 2),
                "max_prb": round(float(subset["Downlink PRB Utilization"].max()), 2),
                "min_prb": round(float(subset["Downlink PRB Utilization"].min()), 2)
            }

    # Calculate summary statistics
    summary_stats = {
        "total_cells": len(all_cells),
        "overall_avg_prb": round(float(df["Downlink PRB Utilization"].mean()), 2),
        "overall_max_prb": round(float(df["Downlink PRB Utilization"].max()), 2),
        "overall_min_prb": round(float(df["Downlink PRB Utilization"].min()), 2)
    }

    return {
        "all_cells": all_cells,
        "cells_data": cells_data,
        "summary_stats": summary_stats
    }

# Store the DataFrame
_dataframe_store['df'] = df



In [45]:
import json
@tool
def save_threshold_recommendations(thresholds_json: str) -> dict:
    """
    Saves the threshold recommendations to a structured format.

    Args:
        thresholds_json: JSON string containing threshold recommendations.
                        Format: [{"cell_id": "Cell_01", "threshold": 40, "rationale": "..."}, ...]

    Returns:
        Confirmation message
    """
    try:
        thresholds = json.loads(thresholds_json)

        # Save to global store
        threshold_df = pd.DataFrame(thresholds)
        _dataframe_store['thresholds'] = threshold_df

        # Save to CSV
        threshold_df.to_csv('cell_thresholds.csv', index=False)

        return {
            "status": "success",
            "message": f"Saved {len(thresholds)} threshold recommendations",
            "file": "cell_thresholds.csv"
        }
    except Exception as e:
        return {
            "status": "error",
            "message": str(e)
        }

In [46]:
# Create agent with enhanced instructions
from agno.agent import Agent
from agno.models.google import Gemini

prb_agent = Agent(
    name="PRB Network Optimization Agent",
    role="Telecom Network PRB utilization expert and optimization specialist",
    model=Gemini(
        id="gemini-2.0-flash",
        api_key=userdata.get('GOOGLE_API_KEY')
    ),
    tools=[get_all_cells_prb_timeseries,save_threshold_recommendations],
    instructions="""
    You are an expert telecom network optimizer analyzing PRB utilization across all cells.

    Your task:
    1) Call get_all_cells_prb_timeseries() to retrieve data for ALL cells

    2) For EACH cell, analyze:
       - Peak PRB hours (highest utilization periods)
       - Low traffic hours (lowest utilization periods)
       - Average daily utilization pattern
       - Congestion indicators (PRB > 70% indicates congestion)
       - Unusual spikes or anomalies

    3) Determine optimal THRESHOLD values:
       - Calculate a PRB threshold for each cell below which it can be switched off
       - Consider: minimum PRB values, traffic patterns, redundancy requirements
       - Thresholds should typically be between 10-30% PRB utilization
       - Account for time-of-day patterns (e.g., nighttime vs daytime)

    4) Provide a comprehensive report with:
       - Summary of all cells analyzed
       - Individual cell analysis with peak/low hours
       - **CRITICAL**: A table/list showing each cell with its recommended switch-off threshold
       - Optimization recommendations (load balancing, cell switching strategies)
       - Potential energy savings estimates

    5) Format the threshold recommendations clearly as:
       Cell ID | Recommended Threshold (%) | Rationale

    6) **IMPORTANT**: After your analysis, call save_threshold_recommendations() with a JSON array containing:
       [
         {"cell_id": "Cell_01", "threshold": 40, "rationale": "High utilization..."},
         {"cell_id": "Cell_02", "threshold": 15, "rationale": "Low utilization..."},
         ...
       ]
    Be specific with numerical thresholds for each cell based on the actual data patterns.
    Always call save_threshold_recommendations() to store the structured data
    """,
    markdown=True,
)

# Run the agent


In [47]:
result = prb_agent.run(
    """Analyze PRB utilization for all cells in the network.

    For each cell, determine:
    1. Traffic patterns and peak/low hours
    2. Optimal PRB threshold value for switching off the cell during low utilization
    3. Provide a clear table with cell IDs and their recommended thresholds
    4. After your analysis, save the thresholds using save_threshold_recommendations().
    Be specific with the threshold values based on the actual data."""
)

print(result.content)

Okay, I have analyzed the PRB utilization data for all ten cells. Here's a summary of my findings and recommendations:

**Summary of Analysis**

The PRB utilization varies significantly across the cells. Some cells, like Cell_01 and Cell_05, consistently have higher utilization, suggesting they serve areas with higher traffic demand. Others, such as Cell_03, Cell_04, and Cell_08, show lower utilization, particularly during off-peak hours. This indicates potential for energy savings through cell switching.

**Individual Cell Analysis and Threshold Recommendations**

Below is a detailed analysis of each cell, including peak/low hours and my recommended switch-off threshold:

| Cell ID | Peak Hours (PRB)          | Low Hours (PRB)         | Avg. PRB | Max PRB | Min PRB | Recommended Threshold (%) | Rationale                                                                                   |
| :-------- | :-------------------------- | :-------------------------- | :------- | :------ | :---

In [48]:
threshold_df=pd.read_csv('cell_thresholds.csv')
threshold_df.head(10)

,cell_id,threshold,rationale
0,Cell_01,40,High utilization; threshold set above minimum ...
1,Cell_02,15,Low utilization overnight; threshold allows sw...
2,Cell_03,15,Low utilization overnight.
3,Cell_04,12,Very low minimum utilization; aggressive thres...
4,Cell_05,35,"High average utilization, but a significant dr..."
5,Cell_06,20,Moderate utilization; threshold allows switchi...
6,Cell_07,30,Consistent utilization; moderate threshold to ...
7,Cell_08,12,Very low minimum utilization.
8,Cell_09,25,Moderate average utilization with a decent range.
9,Cell_10,25,Similar utilization pattern and recommendation...
